## Assignment 4 : web scraping 
#### Name : Amit Gido
#### ID : 315754606

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re


In [2]:
URL = "https://www.madlan.co.il/for-sale/חיפה-ישראל"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
page = requests.get(URL, headers=headers)


In [13]:
if page.status_code == 200:
    soup = BeautifulSoup(page.content, "html.parser")
    apartments = []
    links = soup.find_all('a', href=True)
    for link in links:
        href = link['href']
        if href.startswith('/listings'):
            apartment_url = f"https://www.madlan.co.il{href}"
            print(apartment_url)
            apartment_page = requests.get(apartment_url, headers=headers)
            if apartment_page.status_code == 200:
                apartment_soup = BeautifulSoup(apartment_page.content, "html.parser")

                primary_address = apartment_soup.find('h1', {'data-auto': 'primary_address_text'})
                if primary_address is None:
                    continue

                address_parts = primary_address.text.split()
                if len(address_parts) < 2:
                    continue

                street = ' '.join(address_parts[:-1])
                number_in_street = ''.join(filter(str.isdigit, address_parts[-1]))
                if number_in_street:
                    number_in_street = int(number_in_street)
                else:
                    number_in_street = None

                secondary_address = apartment_soup.find('div', {'data-auto': 'secondary_address_text'}).text
                if ',' in secondary_address:
                    city_area, city = secondary_address.split(', ')
                else:
                    city_area = ''
                    city = secondary_address

                type_ = apartment_soup.find('div', {'data-auto': 'business-class'}).text
                room_number = apartment_soup.find('div', {'data-auto': 'beds-count'}).text

                floor_out_of = apartment_soup.find('div', {'data-auto': 'floor'}).text
                area = apartment_soup.find('div', {'data-auto': 'area'}).text

                condition = apartment_soup.find('div', {'data-auto': 'unit-general-condition-value'}).text
                entranceDate = apartment_soup.find('div', {'data-auto': 'unit-availability-value'}).text
                furniture = apartment_soup.find('div', {'data-auto': 'unit-furniture-details-value'}).text

                description_elem = apartment_soup.find('div', {'class': 'css-v1qjdi ebqee3y2'})
                description = description_elem.text if description_elem else ''

                price_text = apartment_soup.find('div', {'data-auto': 'current-price'}).text
                price = int(re.sub(r'\D', '', price_text))
                specification = apartment_soup.find('div', {'data-auto' : "amenities-block"}).get_text()
                hasElevator = 'יש מעלית' in specification
                hasParking = "יש חניה" in specification
                hasBars = "יש סורגים" in specification
                hasStorage = 'יש מחסן' in specification
                hasAirCondition = "יש מיזוג אויר" in specification
                hasBalcony = "יש מרפסת" in specification
                hasMamad = "יש ממ" in specification
                handicapFriendly = "לא נגיש לנכים" not in specification
                num_of_images =len(apartment_soup.find_all('div' ,{'data-auto':"bulletin-image"}))
                

                apartments.append({
                    'City': city,
                    'type': type_,
                    'room_number': room_number,
                    'Area': area,
                    'Street': street,
                    'number_in_street': number_in_street,
                    'city_area': city_area,
                    'price': price,
                    'floor_out_of': floor_out_of,
                    'condition': condition,
                    'entranceDate': entranceDate,
                    'furniture': furniture,
                    'description': description,
                    'hasElevator': hasElevator,
                    'hasParking' : hasParking,
                    'hasBars' : hasBars,
                    'hasStorage' : hasStorage,
                    'hasAirCondition' : hasAirCondition,
                    'hasBalcony' : hasBalcony,
                    'hasMamad' : hasMamad,
                    'handicapFriendly' : handicapFriendly,
                    'num_of_images': num_of_images })
                
            else:
                print(f'Error status code {apartment_page.status_code} link {apartment_url}')
                break
                

    df = pd.DataFrame(apartments)
    df
    
else:
    print(f'Error status code {page.status_code}')
    

https://www.madlan.co.il/listings/JduTIv3ZPTW
https://www.madlan.co.il/listings/jpCiMWULn85
https://www.madlan.co.il/listings/g0IZ6QbrPIE
https://www.madlan.co.il/listings/cXsCPN9PpmB
https://www.madlan.co.il/listings/N06JFicphWu
https://www.madlan.co.il/listings/ilcnkBZ4aQg
https://www.madlan.co.il/listings/cKm5oL9e7cv
https://www.madlan.co.il/listings/gWPtowyzlET
https://www.madlan.co.il/listings/PO9X7WJlCPl
https://www.madlan.co.il/listings/KbLOIYKYCw8
https://www.madlan.co.il/listings/fYt0OnN5GvZ
https://www.madlan.co.il/listings/egcj72Wq04e
https://www.madlan.co.il/listings/7cW30FFbhyk
https://www.madlan.co.il/listings/jj1RVXdmbU8
https://www.madlan.co.il/listings/kM9IHU5KlJH
https://www.madlan.co.il/listings/dQDfDpdnReZ
Error status code 403 link https://www.madlan.co.il/listings/dQDfDpdnReZ


In [14]:
df.to_excel('real_estate_output.xlsx')


<ipython-input-14-3e1d22e2fa85>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel('real_estate_output.xlsx')


In [15]:
df


,City,type,room_number,Area,Street,number_in_street,city_area,price,floor_out_of,condition,...,description,hasElevator,hasParking,hasBars,hasStorage,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,num_of_images
0,חיפה,דירת גן,5 חד׳,"225 מ""ר",שמשון,35.0,אחוזה,3260000,קומת קרקע,חדש,...,"מפרטי, ללא תיווך, באחוזה, נדירה - דירת גן בת 5...",True,False,False,False,True,True,True,True,10
1,חיפה,דירה,3.5 חד׳,"100 מ""ר",הלל,36.0,הדר הכרמל,1195000,קומה 3 מתוך 3,משופץ,...,,False,False,True,False,True,True,False,False,4
2,חיפה,פנטהאוז,4 חד׳,"125 מ""ר",רציף פנחס,33.0,בת גלים,4400000,קומה 5 מתוך 5,שמור,...,"דירת גג דופלקס (קומות 4 ו- 5) כ- 90 מ""ר בבנין ...",True,False,False,True,True,True,True,False,10
3,חיפה,דירת גג,3.5 חד׳,"78 מ""ר",פבזנר,29.0,הדר הכרמל,2050000,קומה 9 מתוך 9,חדש,...,*לא למתווחים אז בבקשה מתווחים לא להתקשר * דירת...,True,False,False,True,True,False,True,True,10
4,חיפה,דירה,3 חד׳,"86 מ""ר",הס,32.0,הדר עליון,1870000,קומה 3 מתוך 4,חדש,...,"דירת 3 חד' חדשה למכירה! בפרויקט הבוטיק ""הס 32""...",True,False,False,True,True,True,True,True,19
5,חיפה,פנטהאוז,5.5 חד׳,"192 מ""ר",אלכסנדר ינאי,6.0,כרמליה,4700000,קומה 6 מתוך 6,חדש,...,"מכירה/להשכרה (מחיר לשכירות הוא 13,000 ש""ח ולא ...",False,False,False,False,False,False,False,False,5
6,חיפה,דירה,4 חד׳,"120 מ""ר",שדרות,NaN,מרכז הכרמל,3400000,קומה 19 מתוך 20,שמור,...,במרכז הכרמל ממש בפנורמה דירת 4 מרווחת בבניין ש...,False,False,False,True,True,False,True,True,1
7,חיפה,בית פרטי,6 חד׳,"281 מ""ר",הואדי,9.0,ואדי ניסנאס,4000000,קומת קרקע,שמור,...,רחוב תיירותי באמצע ואדי נסנאס הנכס הכי מרכזי ו...,False,False,True,True,False,False,True,True,3
8,חיפה,דופלקס,4.5 חד׳,"131 מ""ר",סנש חנה,22.0,רמת הדר,2545000,קומה 13 מתוך 14,שמור,...,דירת נוף מקסימה במתחם של שני מגדלים. מיקום מעו...,True,False,False,True,True,True,False,False,8
